In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# General libraries
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Deep learning libraries
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, LeakyReLU, Activation, Lambda, GlobalAveragePooling2D, DepthwiseConv2D, GlobalMaxPooling2D, Conv2DTranspose, GaussianNoise
from keras.layers import Add, Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam, SGD
from keras import regularizers
import tensorflow as tf


# Setting seeds for reproducibility
seed = 232
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
input_path1 = '/content/drive/My Drive/four_class/Fold 3/'
train_dir = input_path1 +"train/"
test_dir = input_path1 +"test/"

# **Generating augmented data **

In [ ]:
def generate_generator_multiple(generator,dir1, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                  target_size = (img_height,img_width),
                                  class_mode = 'categorical',
                                  batch_size = batch_size,
                                  shuffle=True,subset='training')
    while True:
        X1i = genX1.next()
        yield [X1i[0], X1i[0],X1i[0],X1i[0]], X1i[1]  #Yield both images and their mutual label

In [ ]:
def generate_generator_multiple1(generator,dir1, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                  target_size = (img_height,img_width),
                                  class_mode = 'categorical',
                                  batch_size = batch_size,
                                  shuffle=True,subset='validation')
    while True:
        X1i = genX1.next()
        yield [X1i[0], X1i[0],X1i[0],X1i[0]], X1i[1]  

In [ ]:
img_dims = 227
epochs = 30
batch_size = 8

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   
                                   validation_split = 0.2)
#test_val_datagen=ImageDataGenerator(validation_split=0.2,rescale=1./255)
train_gen = train_datagen.flow_from_directory(directory=train_dir,subset='training',target_size=(img_dims, img_dims), batch_size=8,class_mode='categorical',shuffle=True)
    
val_gen = train_datagen.flow_from_directory(directory=train_dir,subset='validation',target_size=(img_dims, img_dims), batch_size=8,class_mode='categorical',shuffle=True)
    
   
    
valinputgenerator=generate_generator_multiple1(generator=train_datagen,
                                           dir1=train_dir,
                                           batch_size=8,
                                           img_height=img_dims,
                                           img_width=img_dims)
traininputgenerator=generate_generator_multiple(generator=train_datagen,
                                           dir1=train_dir,
                                           batch_size=8,
                                           img_height=img_dims,
                                           img_width=img_dims)

In [ ]:
 def process_data(img_dims, batch_size):
    test_data = []
    test_labels = []

    #This code assumes that the name of the folders inside the train folder, test folder and validation folder are named as "PNEUMONIA", "NORMAL" and "COVID19"
    for cond in ['/NORMAL/', '/Bacterial/', '/Viral/']:#####################
        for img in (os.listdir(input_path1 + 'test' + cond)):
            img = cv2.imread(input_path1+'test'+cond+img, 0) #We are taking image in grayscale form. 
            img = cv2.resize(img, (img_dims, img_dims)) #Resizing to fit the train image size
            img = np.dstack([img, img, img])  #Feinting color image channel
            img = img.astype('float32') / 255.0
            if cond=='/NORMAL/':
                label = 1
            elif cond=='/Bacterial/':
                label = 0
            else:
              label = 2 
            test_data.append(img)
            test_labels.append(label)
        
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)

    return test_data, test_labels   

In [ ]:

test_data, test_labels = process_data(img_dims, batch_size)
test_data.shape

# **Non-Trainable Model for four paths**


In [ ]:
from keras.applications import MobileNet, DenseNet201, NASNetMobile,InceptionV3,Xception

def create_cnn(img_dims):
  model =  Xception(input_shape=(img_dims, img_dims, 3), weights=None, include_top=False,pooling='avg')

  #x = GlobalAveragePooling2D(name='GlobalAvgPool')(model.output)
  x = Dense(256, activation='relu', name='Bottleneck1')(model.output)
  x = Dense(128, activation='relu', name='Bottleneck2')(x)
  x=Dropout(.2)(x)
  x = Dense(2, activation='softmax')(x)

  model_gen = Model(model.input, x)

  return model_gen


# **Trainable Main Model**

In [ ]:
from keras.applications import MobileNet, DenseNet121, NASNetMobile,InceptionV3

def create_cnn1(img_dims):
  model =  Xception(input_shape=(img_dims, img_dims, 3), weights='imagenet', include_top=False,pooling='avg')
  
  for layers in model.layers[:]:
     layers.trainable = True

  #x = GlobalAveragePooling2D(name='GlobalAvgPool')(model.output)
  x = Dense(256, activation='relu', name='Bottleneck1')(model.output)
  x = Dense(128, activation='relu', name='Bottleneck2')(x)
  x=Dropout(.2)(x)
  x = Dense(2, activation='softmax')(x)

  model_gen = Model(model.input, x)
  
  

  return model_gen


In [ ]:
#normpneu_model_name = 'normpneu_FOLD 3 augment' 
normbact_model_name = 'xnormbact_FOLD 3 augment' #You have to change Fold Name Here...................
normvir_model_name = 'xnormvir_FOLD 3 augment'  #You have to change Fold Name Here...................
bactvir_model_name = 'xbactvir_FOLD 3 augment'  #You have to change Fold Name Here...................

#normpneu_model = create_cnn(img_dims)
normbact_model = create_cnn(img_dims)
normvir_model = create_cnn(img_dims)
bactvir_model = create_cnn(img_dims)

##for layer in normpneu_model.layers:
 # layer._name = layer._name + 'normpneu'

for layer in normbact_model.layers:
  layer._name = layer._name + 'normbact'

for layer in normvir_model.layers:
  layer._name = layer._name + 'normvir'

for layer in bactvir_model.layers:
  layer._name = layer._name + 'bactvir'




opt = Adam(lr=0.0001)
#....................We need to tune weights here.......................#
normbact_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
weight_path1 = '/content/drive/My Drive/Results/Weight/baterialvsNormal_xceptionnet.h5'#...................Change it As necessary....................#'################################
normbact_model.load_weights(weight_path1)



#normpneu_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#weight_path2 = '/content/drive/My Drive/Results/Weight/2class_DenseNet201_normpneu_dataaug_new.h5'#...................Change it As necessary....................#'################################
#normpneu_model.load_weights(weight_path2)

normvir_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
weight_path3 = '/content/drive/My Drive/Results/Weight/XceptionnetviralvsNormal_xceptionnet.h5'#...................Change it As necessary....................#'################################
normvir_model.load_weights(weight_path3)

bactvir_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
weight_path4 = '/content/drive/My Drive/Results/Weight/bacterialvsViral_Xceptionnet.h5'#...................Change it As necessary....................#'################################
bactvir_model.load_weights(weight_path4)


In [ ]:
trainable_model_name = 'xtrainable_FOLD 3 augment'  #You have to change Fold Name Here...................

trainable_model = create_cnn1(img_dims)

for layer in trainable_model.layers:
  layer._name = layer._name + 'trainable'



opt = Adam(lr=0.0001)
#....................We need to tune weights here.......................#
trainable_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])




trainable_model.summary()

# FOR ENSEMBLE MODEL

In [ ]:
#normpneu_feat = Model(normpneu_model.input, normpneu_model.layers[-5].output)
#x1=(Dense(128,activation='relu'))(normpneu_feat.output)
#x1=(Dense(128,activation='relu'))(x1)
#model1 = Model(normpneu_feat.input,normpneu_feat.output )

normbact_feat = Model(normbact_model.input, normbact_model.layers[-5].output)
#x2=(Dense(256,activation='relu'))(normbact_feat.output)
#x2=(Dense(128,activation='relu'))(x2)
model2 = Model(normbact_feat.input, normbact_feat.output)

normvir_feat = Model(normvir_model.input, normvir_model.layers[-5].output)
#x3=(Dense(256,activation='relu'))(normvir_feat.output)
#x3=(Dense(128,activation='relu'))(x3)
model3 = Model(normvir_feat.input, normvir_feat.output)

bactvir_feat = Model(bactvir_model.input, bactvir_model.layers[-5].output)
#x4=(Dense(512,activation='relu'))(bactvir_feat.output)
#x4=(Dense(128,activation='relu'))(x4)
model4 = Model(bactvir_feat.input, bactvir_feat.output)

trainable_feat = Model(trainable_model.input, trainable_model.layers[-5].output)
#model_first = Model(model.inputs, model.layers[-3].output)

# First we will try freezing the layers and then feature concatenating. Later we can try training the whole network
#for layer in model1.layers[:]:
  #layer.trainable = True

for layer in model2.layers[:]:
  layer.trainable = True 

for layer in model3.layers[:]:
  layer.trainable = True

for layer in model4.layers[:]:
  layer.trainable = True

#for layer in trainable_feat.layers[:]:
 # layer.trainable = False  

for layer in trainable_feat.layers[:]:
  layer.trainable = True  

x = Concatenate()([trainable_feat.output,model2.output,model3.output,model4.output])
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(.2)(x)
x = Dense(3, activation='softmax')(x)

model = Model([trainable_feat.input,model2.input,model3.input,model4.input], x)

#....................We need to tune weights here.......................#
opt=Adam(lr=.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
######## THIS IS FOR COMBINED MODEL ########
weight_save_path = '/content/drive/My Drive/Results/Weight/'
model_name = '3class_Xception_ensemble_scheme 1_fold 3_fine tune'   #You have to change Fold Name Here...................

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, verbose=2, mode='max', min_lr=0.00001)

checkpoint = ModelCheckpoint(weight_save_path + model_name+ '.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True)
hist = model.fit_generator(traininputgenerator,epochs=25, steps_per_epoch=train_gen.samples // batch_size,validation_data=valinputgenerator,validation_steps=val_gen.samples//batch_size,use_multiprocessing=True,callbacks=[checkpoint,lr_reduce])


In [ ]:
model.save_weights(weight_save_path + model_name + '.h5')

In [ ]:
from sklearn.metrics import classification_report
from tensorflow.keras.utils import plot_model

y_pred = model.predict([test_data,test_data,test_data,test_data])

y_pred_bool = np.argmax(y_pred, axis=1)
print(np.unique(y_pred_bool))
report = classification_report(test_labels, y_pred_bool, output_dict=True)
print(classification_report(test_labels, y_pred_bool))


############## Make Sure to Change this ####################
plot_save_path = '/content/drive/My Drive/Results/ModelPlot/'
hist_save_path = '/content/drive/My Drive/Results/History/'
result_save_path = '/content/drive/My Drive/Results/Result/'
confusion_matrix_save_path = '/content/drive/My Drive/Results/Confusion Matrix/'


hist_df = pd.DataFrame(hist.history)
hist_csv_file = hist_save_path+model_name+'_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

plot_model(model, plot_save_path + model_name+'.png', show_shapes=True)

result_df = report
result_df = pd.DataFrame(result_df).transpose()

print(result_df)

result_df.to_csv(result_save_path + model_name + '_result.csv')

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from seaborn import heatmap
from matplotlib import pyplot as plt

preds = np.argmax(model.predict([test_data,test_data,test_data,test_data]), axis=1)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
cm_norm = confusion_matrix(test_labels, np.round(preds), normalize='true')

print('CONFUSION MATRIX ------------------')

ax = heatmap(cm, cmap='Blues', linecolor='lightblue',linewidths=.5,annot=True,annot_kws={'size': 12,"weight": "bold"}, xticklabels=['BACTERIAL','NORMAL','VIRAL'], yticklabels=['BACTERIAL','NORMAL','VIRAL'], square=True, fmt='d')

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10,rotation=0)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10,rotation=0)

plt.savefig(confusion_matrix_save_path+model_name+'.png',dpi=500)
plt.show()

ax = heatmap(cm_norm, cmap='Blues', annot=True,linecolor='lightblue',linewidths=.5,annot_kws={'size': 12,"weight": "bold"}, xticklabels=['BACTERIAL','NORMAL','VIRAL'], yticklabels=['BACTERIAL','NORMAL','VIRAL'], square=True, fmt='.2f')

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10,rotation=0)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10,rotation=0)

plt.savefig(confusion_matrix_save_path+model_name+'_normalized.png',dpi=500)